In [1]:
import pandas as pd
import graph as helper
import networkx as nx
import numpy as np
import multiprocessing as mp
import itertools
from datetime import datetime
import math
import features
import normalize_wiki_file

from sklearn.linear_model import LinearRegression

In [8]:
data = pd.read_csv('wiki-features-1000-lines.tsv', sep = "\t")  # load data set
data = data.drop('K', axis = 1)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('edge_sign',axis=1),data['edge_sign'], test_size=0.10, random_state=101)

In [16]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

C:\Users\omyiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [17]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

          -1       0.92      0.44      0.59        25
           0       1.00      1.00      1.00     16417
           1       0.73      0.54      0.62        83

    accuracy                           1.00     16525
   macro avg       0.88      0.66      0.74     16525
weighted avg       1.00      1.00      1.00     16525

